In [ ]:
%cd ..

In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='omniglot_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

Files already downloaded and verified


In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = config.init_obj('optimizer', pyro.optim)

In [11]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader)

In [12]:
trainer.train()

Train Epoch: 1 [128/17352 (1%)] Loss: 12556.362305
Train Epoch: 1 [1536/17352 (9%)] Loss: 11330.934570
Train Epoch: 1 [2944/17352 (17%)] Loss: 4539.952148
Train Epoch: 1 [4352/17352 (25%)] Loss: 4938.904297
Train Epoch: 1 [5760/17352 (33%)] Loss: -5405.991699
Train Epoch: 1 [7168/17352 (41%)] Loss: -6055.822266
Train Epoch: 1 [8576/17352 (49%)] Loss: -3515.976807
Train Epoch: 1 [9984/17352 (58%)] Loss: -3699.166992
Train Epoch: 1 [11392/17352 (66%)] Loss: -11949.865234
Train Epoch: 1 [12800/17352 (74%)] Loss: -12269.368164
Train Epoch: 1 [14208/17352 (82%)] Loss: -12171.238281
Train Epoch: 1 [15616/17352 (90%)] Loss: -11543.471680
Train Epoch: 1 [17024/17352 (98%)] Loss: -12704.007812
    epoch          : 1
    loss           : -4358.008477603688
    val_loss       : -10328.759169578552
Train Epoch: 2 [128/17352 (1%)] Loss: -13932.857422
Train Epoch: 2 [1536/17352 (9%)] Loss: -14605.006836
Train Epoch: 2 [2944/17352 (17%)] Loss: -12624.768555
Train Epoch: 2 [4352/17352 (25%)] Loss: -13